In [12]:
%pylab inline
import pandas as pd
import sklearn as skl
import scipy
import math
import sys
from collections import defaultdict
from gensim import models

model = models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False, no_header=True)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [13]:
document = pd.read_excel("../News_SGU_31077_Processed_1.xlsx")
texts = document["News_Tokens"].apply(lambda x: x.split(" ")).explode().unique()

In [14]:
results = []
skipped=0
for word in texts:
    try:
        # Получаем 10 ближайших слов
        similar_words = model.most_similar(word, topn=10)
        results.append([word, similar_words])
    except KeyError:
        # Если слово не в модели, пропускаем его
        skipped += 1
        continue

df_results = pd.DataFrame(results, columns=["Word", "Most_Similar_Word"])
print(f"Пропустили {skipped} слов")
df_results.to_csv("sim-words10-glove.csv", index=False)

Пропустили 45130 слов
